In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader
import seaborn
from pandas.plotting import register_matplotlib_converters
import talib.abstract as ta
from fracdiff.sklearn import FracdiffStat, Fracdiff, fracdiff
import numpy as np

In [2]:
import yfinance

In [14]:
df = yfinance.download("AAPL", start="2021-01-01", end="2022-01-01").reset_index()
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,127.679977,143301900
1,2021-01-05,128.889999,131.740005,128.429993,131.009995,129.258606,97664900
2,2021-01-06,127.720001,131.050003,126.379997,126.599998,124.907562,155088000
3,2021-01-07,128.360001,131.630005,127.860001,130.919998,129.169800,109578200
4,2021-01-08,132.429993,132.630005,130.229996,132.050003,130.284729,105158200
...,...,...,...,...,...,...,...
247,2021-12-27,177.089996,180.419998,177.070007,180.330002,179.016129,74919600
248,2021-12-28,180.160004,181.330002,178.529999,179.289993,177.983688,79144300
249,2021-12-29,179.330002,180.630005,178.139999,179.380005,178.073059,62348900
250,2021-12-30,179.470001,180.570007,178.089996,178.199997,176.901627,59773000


In [15]:
df["APO"] = ta.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0)

macd, macdsignal, macdhist = ta.MACD(df["Close"], fastperiod=12, slowperiod=26, signalperiod=9)
df["macd"] = macd
df["macdsignal"] = macdsignal
df["macdhist"] = macdhist

df["PPO"] = ta.PPO(df["Close"], fastperiod=12, slowperiod=26, matype=0)

In [16]:
df.dropna(inplace=True)

In [17]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,APO,macd,macdsignal,macdhist,PPO
33,2021-02-22,128.009995,129.720001,125.599998,126.000000,124.501350,103916400,-0.863270,-0.192117,1.809840,-2.001957,-0.642263
34,2021-02-23,123.760002,126.709999,118.389999,125.860001,124.363022,158273000,-1.706796,-0.706261,1.306620,-2.012881,-1.270945
35,2021-02-24,124.940002,125.559998,122.230003,125.349998,123.859085,111039900,-2.588782,-1.141716,0.816953,-1.958668,-1.928694
36,2021-02-25,124.680000,126.459999,120.540001,120.989998,119.550941,148199500,-3.652373,-1.817679,0.290026,-2.107705,-2.726434
37,2021-02-26,122.589996,124.849998,121.199997,121.260002,119.817726,164560400,-4.467308,-2.305027,-0.228984,-2.076043,-3.345113
...,...,...,...,...,...,...,...,...,...,...,...,...
247,2021-12-27,177.089996,180.419998,177.070007,180.330002,179.016129,74919600,6.345192,5.051209,5.174498,-0.123288,3.758949
248,2021-12-28,180.160004,181.330002,178.529999,179.289993,177.983688,79144300,5.915512,5.102919,5.160182,-0.057263,3.487382
249,2021-12-29,179.330002,180.630005,178.139999,179.380005,178.073059,62348900,5.185449,5.092459,5.146637,-0.054178,3.043990
250,2021-12-30,179.470001,180.570007,178.089996,178.199997,176.901627,59773000,4.729679,4.932099,5.103730,-0.171631,2.765714


In [18]:
cols = [c for c in df.columns if c != "Date"]
new_cols = ["frac_" + c for c in cols]
cols, new_cols

(['Open',
  'High',
  'Low',
  'Close',
  'Adj Close',
  'Volume',
  'APO',
  'macd',
  'macdsignal',
  'macdhist',
  'PPO'],
 ['frac_Open',
  'frac_High',
  'frac_Low',
  'frac_Close',
  'frac_Adj Close',
  'frac_Volume',
  'frac_APO',
  'frac_macd',
  'frac_macdsignal',
  'frac_macdhist',
  'frac_PPO'])

In [19]:
window=100
fs = FracdiffStat(window=window, mode="same", precision=0.1, upper=2.0)

In [20]:
df[new_cols] = fs.fit_transform(df[cols])
df

,Date,Open,High,Low,Close,Adj Close,Volume,APO,macd,macdsignal,...,frac_High,frac_Low,frac_Close,frac_Adj Close,frac_Volume,frac_APO,frac_macd,frac_macdsignal,frac_macdhist,frac_PPO
33,2021-02-22,128.009995,129.720001,125.599998,126.000000,124.501350,103916400,-0.863270,-0.192117,1.809840,...,129.720001,125.599998,126.000000,124.501350,103916400.0,-0.863270,-0.192117,1.809840,-2.001957,-0.642263
34,2021-02-23,123.760002,126.709999,118.389999,125.860001,124.363022,158273000,-1.706796,-0.706261,1.306620,...,110.494999,94.840000,102.235001,101.019019,158273000.0,-1.706796,-0.610202,1.306620,-2.012881,-1.270945
35,2021-02-24,124.940002,125.559998,122.230003,125.349998,123.859085,111039900,-2.588782,-1.141716,0.816953,...,102.627185,90.464691,92.153592,91.057517,111039900.0,-2.588782,-0.764570,0.816953,-1.958668,-1.928694
36,2021-02-25,124.680000,126.459999,120.540001,120.989998,119.550941,148199500,-3.652373,-1.817679,0.290026,...,99.401757,82.823711,82.101297,81.124783,148199500.0,-3.652373,-1.146531,0.290026,-2.107705,-2.726434
37,2021-02-26,122.589996,124.849998,121.199997,121.260002,119.817726,164560400,-4.467308,-2.305027,-0.228984,...,94.658242,79.775704,79.156963,78.215461,164560400.0,-4.467308,-1.201827,-0.228984,-2.076043,-3.345113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2021-12-27,177.089996,180.419998,177.070007,180.330002,179.016129,74919600,6.345192,5.051209,5.174498,...,98.441695,71.293401,73.227215,72.711081,74919600.0,6.345192,0.654306,5.174498,-0.123288,3.758949
248,2021-12-28,180.160004,181.330002,178.529999,179.289993,177.983688,79144300,5.915512,5.102919,5.160182,...,98.707157,71.959335,71.128212,70.626697,79144300.0,5.915512,0.603261,5.160182,-0.057263,3.487382
249,2021-12-29,179.330002,180.630005,178.139999,179.380005,178.073059,62348900,5.185449,5.092459,5.146637,...,97.542880,70.839166,70.873674,70.373536,62348900.0,5.185449,0.546368,5.146637,-0.054178,3.043990
250,2021-12-30,179.470001,180.570007,178.089996,178.199997,176.901627,59773000,4.729679,4.932099,5.103730,...,97.265981,70.416806,69.376885,68.887149,59773000.0,4.729679,0.374281,5.103730,-0.171631,2.765714


In [21]:
fs.d_

array([0.125 , 0.125 , 0.1875, 0.1875, 0.1875, 0.    , 0.    , 0.5   ,
       0.    , 0.    , 0.    ])

In [12]:
dict(zip(cols, fs.d_))

{'Open': 0.125,
 'High': 0.125,
 'Low': 0.1875,
 'Close': 0.1875,
 'Adj Close': 0.1875,
 'Volume': 0.0,
 'APO': 0.0,
 'macd': 0.5,
 'macdsignal': 0.0,
 'macdhist': 0.0,
 'PPO': 0.0}

In [24]:
temp_df = df.iloc[:, 1:4]
temp_df

,Open,High,Low
33,128.009995,129.720001,125.599998
34,123.760002,126.709999,118.389999
35,124.940002,125.559998,122.230003
36,124.680000,126.459999,120.540001
37,122.589996,124.849998,121.199997
...,...,...,...
247,177.089996,180.419998,177.070007
248,180.160004,181.330002,178.529999
249,179.330002,180.630005,178.139999
250,179.470001,180.570007,178.089996


In [39]:
fracdiff.fdiff(temp_df, [.5, .3, .1])

AttributeError: 'list' object has no attribute 'is_integer'

In [37]:
fd = Fracdiff(d=0.2, window=window, mode="same")

In [38]:
fd.fit_transform(df[cols])

array([[ 1.28009995e+02,  1.29720001e+02,  1.25599998e+02, ...,
         1.80984001e+00, -2.00195739e+00, -6.42262670e-01],
       [ 9.81580032e+01,  1.00765999e+02,  9.32699997e+01, ...,
         9.44651776e-01, -1.61248944e+00, -1.14249215e+00],
       [ 8.99472025e+01,  8.98403976e+01,  8.85040036e+01, ...,
         4.10841526e-01, -1.39593561e+00, -1.62312429e+00],
       ...,
       [ 6.67798499e+01,  6.67750518e+01,  6.65267781e+01, ...,
         2.34210280e+00, -9.99936063e-02,  8.56603776e-01],
       [ 6.65653835e+01,  6.64454556e+01,  6.60928923e+01, ...,
         2.28683907e+00, -2.21054953e-01,  7.05142060e-01],
       [ 6.48525923e+01,  6.48656272e+01,  6.49574905e+01, ...,
         2.19977277e+00, -3.48265197e-01,  6.02366086e-01]])

In [36]:
df.shape

(219, 23)

In [ ]:
fd_params["last_generated"] = "2222"
fd_params["params"] = dict(zip(cols, fs.d_))
fd_params

In [ ]:
df[["macd", "frac_macd"]].plot()

In [ ]:
df.apply(lambda x: fs.fit_transform(x))

In [ ]:
diff.shape

In [ ]:
df_fd["Open_fd"].plot()

In [ ]:
df["Open"].plot()

In [ ]:
df.select_dtypes("number")

In [ ]:
a = np.random.random(size=(20000,4))

In [ ]:
a_df = pd.DataFrame(a)
a_df

In [ ]:
window=100
fs = FracdiffStat(window=window, mode="same", precision=0.1)

In [ ]:
fd =Fracdiff(d=0.5, window=window, mode="same")

In [ ]:
a

In [ ]:
%%timeit
a_fd = fs.fit_transform(X=a)

In [ ]:
%%timeit
a_df_fd = fs.fit_transform(X=a_df[::10])

In [ ]:
%%timeit
a_df_fd = fs.fit_transform(X=a_df)
a_df.shape, a_df_fd.shape

In [ ]:
a.shape, a_fd.shape

In [ ]:
a_df.shape, a_df_fd.shape

In [ ]:
a_df = a_df.rename(columns={0: "test"})

In [ ]:
a_df.loc[::4, "test"]

In [40]:
import time

In [42]:
l = ["ds", "tr", "uy"]

In [45]:
[c for c in l if "d" not in c and "u" not in c]

['tr']

In [46]:
s = "_long"

In [50]:
"_" in s

True